# Practical Example: Using `sql_connection` for SQL Queries with SQLite

This notebook demonstrates how to establish a connection to an existing database using the OOP-based `sql_connection` module. The goal is to perform safe, read-only queries against a real database without creating or modifying tables.

**Sample database:** `toys_and_models.sqlite` (located in `examples/`)

You can override the default path by defining the environment variable `SQL_CONN_EXAMPLE_DB`.

---

### 🧭 Table of Contents
1. [Requirements](#requirements)
2. [Importing the Module](#importing-the-module)
3. [Connecting to SQLite](#connecting-to-sqlite)
4. [Listing Tables](#listing-tables)
5. [Basic Queries](#basic-queries)
6. [Parameterized Query](#parameterized-query)
7. [Conclusions](#conclusions)

---

## Requirements

Install the package in editable mode from the project root:
```bash
pip install -e .
```

If you want to display results as DataFrames, install **pandas**:
```bash
pip install pandas
```

In [1]:
from sql_connection import get_connector
import os
try:
    import pandas as pd
except ImportError:
    pd = None

## Connecting to SQLite

We first define the path to the database file and establish a connection using the module’s unified API.

In [2]:
# Define database path (can be overridden by environment variable)
db_path = os.environ.get("SQL_CONN_EXAMPLE_DB", "toys_and_models.sqlite")
db_path = os.path.abspath(db_path)

if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database not found: {db_path}")

# Create connector and open connection
conn = get_connector("sqlite", path=db_path)

with conn:
    print("✅ Connected to:", conn.dsn_summary())
    print("Ping:", conn.ping())

## Listing Tables

Retrieve all table names available in the sample database.

In [3]:
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"

if pd:
    display(conn.read_sql(query_tables))
else:
    rows = conn.query(query_tables)
    print("Tables:", rows)

## Basic Queries

Fetch a sample of customer data to validate query execution.

In [4]:
query_customers = "SELECT customerNumber, customerName, country FROM customers LIMIT 10;"

if pd:
    display(conn.read_sql(query_customers))
else:
    rows = conn.query(query_customers)
    for row in rows:
        print(row)

### Query: Top 5 Countries by Customer Count

In [5]:
query_top_countries = """
SELECT country, COUNT(*) AS num_customers
FROM customers
GROUP BY country
ORDER BY num_customers DESC
LIMIT 5;
"""

if pd:
    display(conn.read_sql(query_top_countries))
else:
    rows = conn.query(query_top_countries)
    print("Top countries:", rows)

## Parameterized Query

Parameterization is essential to avoid SQL injection. This example filters customers by country using parameter substitution.

In [6]:
country = "USA"
sql_param = """
SELECT customerNumber, customerName, country
FROM customers
WHERE country = :country
LIMIT 5;
"""

params = {"country": country}

if pd:
    display(conn.read_sql(sql_param, params=params))
else:
    rows = conn.query(sql_param, params=params)
    for r in rows:
        print(r)

## Closing the Connection

Always close your connections to release resources.

In [7]:
conn.close()
print("🔒 Connection closed successfully.")

## Conclusions

- The `sql_connection` module provides a unified, object-oriented API for connecting to multiple SQL engines (SQLite, PostgreSQL, MySQL, etc.).
- It allows developers and analysts to query data seamlessly without rewriting connection logic per engine.
- This example demonstrates safe query execution, context-managed connections, and optional DataFrame integration via pandas.
- Ideal for ETL pipelines, analytics workflows, and reproducible data exploration.